In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# from degree_days import dds
import datetime
from sklearn.model_selection import train_test_split, KFold
from common import compute_rmse_fraction, contri, get_tensor, create_region_df_dfc_static
from create_matrix import *
from tensor_custom_core import *

appliance_index = {appliance: APPLIANCES_ORDER.index(appliance) for appliance in APPLIANCES_ORDER}

APPLIANCES = ['fridge', 'hvac', 'wm', 'mw', 'oven', 'dw']
region = "Austin"
year = 2014

import os

def un_normalize(x, maximum, minimum):
    return (maximum - minimum) * x + minimum

In [4]:
import sys
case=2
a = 3
b = 2
source = 'Austin'
target = 'SanDiego'
constant_use = 'True'
start = 1
stop = 13

In [5]:
source_df, source_dfc, source_tensor, source_static = create_region_df_dfc_static(source, year, start, stop)
target_df, target_dfc, target_tensor, target_static = create_region_df_dfc_static(target, year, start, stop)

# # using cosine similarity to compute L
source_L = get_L(source_static)
target_L = get_L(target_static)

# Seasonal constant constraints
if constant_use == 'True':
    T_constant = np.ones(stop-start).reshape(-1 , 1)
else:
    T_constant = None

In [6]:
tensor_copy = source_tensor.copy()
tensor_copy[:, 1:, :] = np.NaN

In [46]:
def compute_error(HAT, index, domain):
    pred = {}
    for appliance_name, appliance_id in appliance_index.iteritems():
        pred[appliance_name] = []
        pred[appliance_name].append(pd.DataFrame(HAT[:,appliance_id,:], index=index))
#     print pred
    for appliance_name, appliance_id in appliance_index.iteritems():
        pred[appliance_name] = pd.concat(pred[appliance_name]).ix[index]
    
    
#     pred = pd.DataFrame(pred)
    err_all = {}
    err = {}
    for appliance in APPLIANCES_ORDER[1:]:        
        if appliance=="hvac":
            err_all[appliance], err[appliance] = compute_rmse_fraction(appliance, pred[appliance][range(4, 10)], domain)[2:]
        else:   
            err_all[appliance], err[appliance] = compute_rmse_fraction(appliance, pred[appliance], domain)[2:]

    err_app = {}
    for appliance in APPLIANCES_ORDER[1:]:
        if appliance == 'hvac':
            start, end = 5, 11
        else:
            start, end = 1, 13

        error_home = pd.concat([err[appliance][appliance + "_{}".format(start)], 
                           err[appliance][appliance + "_{}".format(start+1)]],axis=1)

        for i in range(start+2, end):
            error_home = pd.concat([error_home, err[appliance][appliance + "_{}".format(i)]], axis = 1)
        app = np.sqrt((error_home**2).mean(axis=1))
        err_app[appliance] = app

    result = (pd.DataFrame(err_app).fillna(0)*pd.Series(contri[domain])).sum(axis=1)
        
        
    return err_all, result

In [78]:
H, A, T, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy,
                                                            source_L,
                                                          a,
                                                          b,
                                                          num_iter=1300,
                                                          lr=0.1, dis=True,
                                                          lam=0,
                                                          T_known=T_constant)

927.359618339
327.132788609
174.115738163


In [79]:
H

array([[ 1.66345507,  1.8757803 ,  1.69049574],
       [ 5.56361928,  5.50529646,  5.50663552],
       [ 1.3197905 ,  1.78687358,  1.804727  ],
       ..., 
       [ 2.73141718,  2.82475159,  2.26333804],
       [ 2.24214654,  2.14861078,  2.76485631],
       [ 2.99715369,  3.21232919,  3.28624526]])

In [11]:
err_all, err_home = compute_error(HATs[-1], source_df.index, source)

/home/yj9xs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


hvac
fridge
mw
dw
wm
oven


In [45]:
source_df.ix[max_index]['aggregate_3']

77.594146728515625

In [17]:
k = ((pd.Series(err_all) * pd.Series(contri['Austin']))).sum()# error = (pd.DataFrame(out)*pd.Series(contri['SanDiego'])).sum(axis=1)

In [21]:
max_index = err_home.argmax()

In [40]:
position = source_df.index.get_loc(max_index)

In [42]:
tensor_copy[position] = source_tensor[position]

In [74]:
import random
random.seed(5)
random_list =  random.sample(range(533), 533)
random_list =[x+1 for x in random_list]

In [77]:
prediction_error = []

tensor_copy = source_tensor.copy()
tensor_copy[:, 1:, :] = np.NaN
H, A, T, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy,
                                                            source_L,
                                                          a,
                                                          b,
                                                          num_iter=1300,
                                                          lr=0.1, dis=False,
                                                          lam=0,
                                                      T_known=T_constant)
err_all, err_home = compute_error(HATs[-1], source_df.index, source)
overall_error = ((pd.Series(err_all) * pd.Series(contri['Austin']))).sum()
print overall_error

for i in range(20):
    max_index = err_home.argmax()
    position = source_df.index.get_loc(max_index)
#     position = random_list[i]
    print position
    tensor_copy[position] = source_tensor[position]
    
    H, A, T, Hs, As, Ts, HATs, costs = learn_HAT_adagrad_graph(case, tensor_copy,
                                                            source_L,
                                                          a,
                                                          b,
                                                          num_iter=1300,
                                                          lr=0.1, dis=False,
                                                          lam=0,
                                                          T_known=T_constant)
    err_all, err_home = compute_error(HATs[-1], source_df.index, source)
    overall_error = ((pd.Series(err_all) * pd.Series(contri['Austin']))).sum()
    print i, overall_error

/home/yj9xs/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


26.1021914905
61
0 22.1222043053
47
1 19.8106416228
476
2 18.4501308997
178
3 16.5123081787
518
4 17.1332797771
494
5 17.0051122383
45
6 16.8313282303
318
7 14.7923720999
131
8 14.7421349938
346
9 14.6333198034
153
10 13.6021757488
59
11 13.596117084
439
12 13.4803772491
425
13 13.5001449042
494
14 13.5001449042
494
15 13.5001449042
494
16 13.5001449042
494
17 13.5001449042
494
18 13.5001449042
494
19 13.5001449042
